In [65]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)

In [66]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [67]:
def model(t_u, w, b):
    return w * t_u + b

In [68]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [69]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

In [70]:
params.grad is None

True

In [71]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

params.grad

tensor([4517.2969,   82.6000])

In [72]:
if params.grad is not None:
    params.grad.zero_()

In [73]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # <1>
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():  # <2>
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [74]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)

Epoch 500, Loss 7.860116
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957697
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

In [84]:
# https://medium.com/@monadsblog/pytorch-backward-function-e5e2b7e60140
import torch
x=torch.ones(2,2, requires_grad=True)
y=x + 2
z = y ** 2
o = z.sum()
print(o)
o.backward()
# o = sum((x+2)*(x+2)) => 
print(x.grad)

tensor(36., grad_fn=<SumBackward0>)
tensor([[6., 6.],
        [6., 6.]])


In [82]:
import torch
x=torch.ones(2,2, requires_grad=True)
y=x + 2
z = y ** 2
z.backward()
# o = z.sum()
# o.backward()
# o = sum((x+2)*(x+2)) => 
print(x.grad)

RuntimeError: grad can be implicitly created only for scalar outputs

In [88]:
# https://medium.com/@monadsblog/pytorch-backward-function-e5e2b7e60140
x = torch.ones(8)  # input tensor
y = torch.zeros(10)  # expected output
W = torch.randn(8, 10, requires_grad=True) # weights
b = torch.randn(10, requires_grad=True) # bias vector
z = torch.matmul(x, W)+b # output
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [89]:
loss.backward()
print(W.grad) #OK
print(b.grad) #OK
print('~' * 40)
print(x.grad)
print(y.grad)
print(z.grad) # WARNING
print(loss.grad) # WARNING

tensor([[1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671e-02, 3.2859e-05, 7.6058e-03, 9.3191e-02,
         1.6223e-02, 6.9811e-02, 1.0298e-02, 6.7130e-02, 9.2292e-02],
        [1.4536e-02, 7.7671

In [90]:
# 输出非标题
x = torch.rand(3, requires_grad=True)
y = x + 2
# y.backward() <---
# RuntimeError: grad can be implicitly 
# created only for scalar outputs
# try ---> y.backward(v) where v is any tensor of length 3
v = torch.rand(3)
y.backward(v)
print(x.grad)

tensor([0.9404, 0.4203, 0.6135])


In [107]:
x = torch.rand(3, requires_grad=True)
y = x + 2
vvv = torch.ones(3)
grad = torch.autograd.grad(outputs=y, inputs=x, grad_outputs=vvv)
print(grad, vvv)

(tensor([1., 1., 1.]),) tensor([1., 1., 1.])


In [96]:
torch.rand(3)

tensor([0.8891, 0.2402, 0.1214])